In [ ]:
# !pip install datasets transformers==4.38.2 peft==0.10.0 accelerate==0.28.0 bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !git clone https://github.com/maszhongming/UniEval.git
# !pip install -r /content/UniEval/requirements.txt

In [ ]:
# import nltk
# nltk.download('punkt_tab')

In [ ]:
# !unzip gpt-2-tuned-ppo-20250507T113025Z-001.zip

Archive:  gpt-2-tuned-ppo-20250507T113025Z-001.zip
  inflating: gpt-2-tuned-ppo/merges.txt  
  inflating: gpt-2-tuned-ppo/README.md  
  inflating: gpt-2-tuned-ppo/adapter_config.json  
  inflating: gpt-2-tuned-ppo/tokenizer.json  
  inflating: gpt-2-tuned-ppo/vocab.json  
  inflating: gpt-2-tuned-ppo/special_tokens_map.json  
  inflating: gpt-2-tuned-ppo/tokenizer_config.json  
  inflating: gpt-2-tuned-ppo/adapter_model.safetensors  


In [ ]:
import os
import torch
import pandas as pd
from tqdm import tqdm
import numpy as np

# Check if CUDA is available for inference
CUDA_AVAILABLE = torch.cuda.is_available()
DEVICE = "cuda" if CUDA_AVAILABLE else "cpu"
print(f"Using device: {DEVICE} for inference")

# ────────────────────────────────────────────────────────────────
# 1) Load the trained model weights
# ────────────────────────────────────────────────────────────────
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from peft import PeftModel

# Path to the saved model weights
MODEL_PATH = r"/content/gpt-2-tuned-ppo"

# Use the same prompt format that was used during training
PROMPT_TEMPLATE = "Summarize the following conversation:\n\n"

# Load the tokenizer and model from saved path
print(f"Loading model from {MODEL_PATH}")
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_PATH)
model = GPT2LMHeadModel.from_pretrained(MODEL_PATH)

# Ensure pad token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"  # Important for decoder-only models
model = model.to(DEVICE)

Using device: cuda for inference
Loading model from /content/gpt-2-tuned-ppo


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [4]:
# ────────────────────────────────────────────────────────────────
# 1) Load BioMedLM model
# ────────────────────────────────────────────────────────────────
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

CUDA_AVAILABLE = torch.cuda.is_available()
DEVICE = "cuda" if CUDA_AVAILABLE else "cpu"
print(f"Using device: {DEVICE} for inference")

# Stanford's BioMedLM (GPT-2 architecture fine-tuned on medical texts)
MODEL_NAME = "stanford-crfm/BioMedLM"
print(f"Loading biomedical model {MODEL_NAME}")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Configure padding (same as GPT-2 setup)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"  # Critical for autoregressive models

# Move to device
model = model.to(DEVICE)

C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Using device: cuda for inference
Loading biomedical model stanford-crfm/BioMedLM


In [6]:
def return_prompt(conv):
    prompt = f"""
    CONVERSATION:
    {conv}
    
    SUMMARY:
    Please generate a medical summary based on the following clinical notes. The summary should include the following sections: Chief Complaint, History of Present Illness, Vitals, Physical Exam, Results, Assessment, Plan, and Instructions.
    Please format the response as plain text, without using markdown or special formatting, and with clear headings for each section, like this:
    
    
    CHIEF COMPLAINT
    A concise statement of the patient's primary concern or reason for visiting the clinic.
    
    HISTORY OF PRESENT ILLNESS
    A detailed narrative about the patient's symptoms, their onset, duration, and any relevant medical history or previous treatments.
    
    VITALS
    Include any relevant vital signs (e.g., oxygen saturation, blood pressure) if available.
    
    PHYSICAL EXAM 
    Summarize the findings from the physical examination, including any notable abnormalities.
    
    RESULTS 
    Summarize the results of any diagnostic tests performed (e.g., lab work, imaging studies).
    
    ASSESSMENT
    The doctor's assessment of the patient's condition or diagnosis.
    
    PLAN
    The treatment plan, including prescribed medications, lifestyle recommendations, and follow-up instructions.
    
    INSTRUCTIONS
    Specific instructions for the patient regarding their treatment plan and follow-up care.
    
    Important Note: If any section lacks relevant information or if specific details are not provided (e.g., vitals are not mentioned, no abnormal findings in the physical exam), omit that section from the generated summary. Only include sections for which there is sufficient information available.
    """

    return prompt

In [ ]:
data = [{'CHIEF COMPLAINT': "The patient's primary concern is high blood pressure, which led to lightheadedness and a fall during a walk. She reports experiencing high blood pressure episodes", 'HISTORY OF PRESENT ILLNESS': 'HPI: The patient, Diane, a 28-year-old female with a history of depression and hypertension, presented for emergency room follow-up after experiencing high', 'MEDICATIONS': 'Meds: Lisinopril 40mg daily for HTN\nMeds: Lisinopril 40mg daily for HTN.\n\n', 'VITALS': 'Vitals: BP 198/100 mmHg, HR not mentioned, Temperature not mentioned, Respiratory rate not mentioned, O2', 'PHYSICAL EXAM': 'PE: Heart - S2/6 systolic ejection murmur, clear lungs bilaterally, trace pitting edema in lower extremities. AB', 'INSTRUCTIONS': 'Instructions: Take lisinopril 40 mg daily. Monitor blood pressure and report through patient portal. If blood pressure remains high, may need to add'}]

In [ ]:
for i in data:
  for j in i:
    print(j)

CHIEF COMPLAINT
HISTORY OF PRESENT ILLNESS
MEDICATIONS
VITALS
PHYSICAL EXAM
INSTRUCTIONS


In [ ]:
instructions = {'CHIEF COMPLAINT': 'Supposed to be a 3-4 word answer summarizing the concern',
                'HISTORY OF PRESENT ILLNESS': "Supposed to be an answer that summarizes patient's symptoms and relevant medical history and previous treatments.",
                'MEDICATIONS': 'List all medications the patient is taking in a bullet point format. Include dosages if provided',
                'VITALS': 'Summarize all the vital signs in bullet points. Dont add extra information',
                'PHYSICAL EXAM': 'Summarize the findings from the physical examination information.',
                'INSTRUCTIONS': 'Summarize any specific instructions for the patient regarding their treatment plan.'}

In [ ]:
for i in data:
  for key, value in i.items():
      print(f"{key}:\n {value}")

CHIEF COMPLAINT:
 The patient's primary concern is high blood pressure, which led to lightheadedness and a fall during a walk. She reports experiencing high blood pressure episodes
HISTORY OF PRESENT ILLNESS:
 HPI: The patient, Diane, a 28-year-old female with a history of depression and hypertension, presented for emergency room follow-up after experiencing high
MEDICATIONS:
 Meds: Lisinopril 40mg daily for HTN
Meds: Lisinopril 40mg daily for HTN.


VITALS:
 Vitals: BP 198/100 mmHg, HR not mentioned, Temperature not mentioned, Respiratory rate not mentioned, O2
PHYSICAL EXAM:
 PE: Heart - S2/6 systolic ejection murmur, clear lungs bilaterally, trace pitting edema in lower extremities. AB
INSTRUCTIONS:
 Instructions: Take lisinopril 40 mg daily. Monitor blood pressure and report through patient portal. If blood pressure remains high, may need to add


In [ ]:
# GENERATION_PROMPT = "Using only the information present in the context given below, generate a response following the Instruction below. Strictly keep your response short."
GENERATION_PROMPT = """You are a clinical assistant. Using ONLY the instruction and information below, generate a structured medical summary. Do not invent information."""
generation_args = {
    "max_new_tokens": 128,          # Keep responses concise
    "do_sample": False,             # Disable sampling (use deterministic decoding)
    "num_beams": 5,                 # More beams = better coherence (but slower)
    "temperature": 0.1,             # Almost deterministic (lower = less random)
    "top_p": 0.85,                  # Slightly tighter than 0.9
    "top_k": 20,                    # Restrict to top 20 tokens per step
    "no_repeat_ngram_size": 4,      # Prevent repeating 4-grams
    "repetition_penalty": 1.5,      # Strongly discourage repetition
    "length_penalty": 0.8,          # Prefer shorter outputs (optional)
}


# for i in tqdm(range(num_batches)):
#     start, end = i*batch_size, min((i+1)*batch_size, num_samples)
#     convs = dialogues[start:end]

# Use the same prompt format as during training
for i in data:
  summary = []

  for prompt_key, prompt in i.items():
      full_prompt = f"{GENERATION_PROMPT}\Instruction:{instructions[prompt_key]}\Information:{prompt}"

      # Tokenize
      enc = tokenizer(
          full_prompt,
          padding=True,
          truncation=True,
          max_length=512,
          return_tensors="pt"
      ).to(DEVICE)

      # Generate with improved parameters matching training
      with torch.no_grad():
          out_ids = model.generate(
          input_ids=enc.input_ids,
          attention_mask=enc.attention_mask,
          **generation_args
      )

      # Decode the generated IDs, skipping the original prompt
      generated_text = tokenizer.decode(
          out_ids[0][enc.input_ids.size(1):],  # Slice from input length to end
          skip_special_tokens=True
      ).strip()  # Remove any leading/trailing whitespace

      text = f"{prompt_key}:\n{generated_text}\n"
      summary.append(text)

  full = "\n".join(summary)
  print(f"\n{full}")

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
Setting `pad_token_id` to `e


CHIEF COMPLAINT:
in the past 12 months \[[@CR1]\].

**Publisher's Note:**

This article is distributed under the terms of the Creative Commons Attribution Noncommercial 4.0 International License (CC BY 4.0 license/policy. The online version of this article can be found at <https://doi.org/10.1186/s13052-016-0098-5017-3-S1>.

The authors would like to thank Dr

HISTORY OF PRESENT ILLNESS:
blood pressure (BP) during her first trimester of pregnancy, she was admitted to the emergency department with sudden onset of fever, headache, nausea, vomiting, abdominal pain, feverishness, diarrhea, coughing, and feverishness \[[@CR1]\].

**Competing Interests:**

The authors declare that they have read and approved the final version of this manuscript has no competing interests

  -------------------- ----------- ---------- --------
  **Symptoms \#1**

MEDICATIONS:
![](http://dx.doi.org/10.1177_2333794X19829243){#F1}

  --------------------
    **Abbreviations:**    \(A\]   
  -2    Abbreviation B

In [ ]:
full_prompt = f"INSTRUCTION: {prompt['instruction']}\n Generate the information according to the above instruction from below conversation. Only include the relevant information in your response please. Keep responses as short as you can while including all important information. Don't include any dialogues from the conversation in your response\ {convo}"